<div dir="rtl" align="center">
    <h1>
        تمرین اول درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl">
    ما در این تمرین ترک "استخراج پرسش و پاسخ" را انتخاب کردیم. در این تمرین قصد داریم از یک جمله فارسی سوالاتی با توجه به روابط علت و معلولی، نقش کلمات در جملات، ارتباط بین واژگان و ... استخراج کنیم. هر سوال به شکل یک دیکشنری پایتون نمایش داده می‌شود که key سوال و value پاسخ به سوال است.
</div>

<div dir="rtl">
    <h2> 1. تمیز کردن ورودی </h2>
    در این بخش، گام‌های ابتدایی برای مراحل آتی را برمی‌داریم. هر جمله که به عنوان ورودی ما داده می‌شود، باید تا حد مناسبی تمیز و ساختاریافته شود تا به عنوان ورودی بخش‌های بعد معتبر باشد. کارهایی مانند نرمال‌سازی و ... در این بخش انجام می‌گردد. از توابع تعریف شده در کتابخانه hazm استفاده می‌کنیم.
</div>

In [1]:
import re
from collections import Counter
import hazm
from hazm import *

In [2]:
def sentence_clean(sentence: str):
    normalizer = Normalizer()
    return normalizer.normalize(sentence)

sentence = ''
print(sentence_clean(sentence))

<div dir="rtl">
    <h2> 2. استخراج ساختار سوالات </h2>
    در این مرحله، به یافتن کلمات کلیدی در جملاتی که امکان استخراج سوالات از آنها وجود دارد پرداختیم. در یک Google Sheet لیست این کلمات، انواع نحوه به کارگیری آنها در جملات، مثال‌هایی از هر یک و هم‌چنین ساختار سوال و جواب و مثال از آنها را مشخص کردیم. 
    هم‌چنین مثال‌هایی از جملات را به عنوان داده‌ی تست یافتیم و سوال‌هایی از آنها استخراج کردیم.
    <a href="https://docs.google.com/spreadsheets/d/1qsCfRVr5RgXYW2qEVbrP3dq7-l3wNngjPAsXrHN89DQ/edit#gid=0" target="_top">فهرست کلمات کلیدی و جملات تست</a><br/>
    سپس در تابع <code>extract_cause_effect</code> از جمله‌ای که به عنوان ورودی داده می‌شود، با توجه به ساختار جمله که ورودی دیگر این تابع و از انواع پرسش و پاسخ است، رابطه‌ی علت و معلولی که ممکن است در جمله باشد، استخراج می‌گردد.<br>
    در دیکشنری <code>all_regexes</code> نوع و ساختار جملاتی که قابلیت طرح سوال از آنها وجود دارد توسط regex طراحی شده است. هم‌چنین ساختار پرسش و پاسخ آنها نیز وجود دارد.<br>
    اگر ساختار regex در جمله موجود بود، بسته به اینکه کدام بخش جمله در سوال و کدام بخش آن در جواب استفاده شود، سوال و جواب از جمله ساخته شده و در دیکشنری خروجی تابع قرار می‌گیرد.
</div>

In [21]:
all_regexes = {"چرایی": {"regex": r"""(?P<effect>.*)[،|؛|] (?P<reason> زیرا|چون|چون\sکه|به\sدلیل\sاینکه) (?P<cause>.*)""", "question": "چرا {effect} ؟", "answer": "زیرا {cause}"}, 
               "سببی": {"regex": r"(?P<effect>.*) (?P<reason> دلیل|مسبب) (?P<cause>.*)", "question": "{effect} باعث چه می شود؟ ", "answer": "باعث {cause}"}
               }

def extract_cause_effects(text: str, key: str):
    """
    extracting cause and effect parts from text based on key regex. creating a Q&A based on key question format and answer format
    Parameters
    ---------
    text: str
        sentence query used to extract question and answers.
    key: str
        key should be one of all_regexes.keys(). sentence format used to extract Q&A
  
  
    Returns
    -------
    result: dict
        includes question and answer sentences.
    """


    key_information = all_regexes.get(key, None)
    if not key_information:
        print("you should specify current key information in all_regexes dict")
        return
    regex = key_information.get("regex")
    first_reg = re.compile(regex)
    x = re.search(first_reg, text)
    if x is not None:
        cause = x.group("cause")
        effect = x.group("effect")
        if not cause:
        # text is not in the format of first part of regex. checking for second part
            cause = x.group("cause2")
            effect = x.group("effect2")
            if not cause or not effect:
                # cause and effects are empty! this shouldn't happen!!!
                print("bad input")
                return None
    else:
        # regex didn't found the structure. 
        return None
    q_format = key_information.get("question")
    a_format = key_information.get("answer")  
    result = {"question": sentence_clean(q_format.format(effect=effect)), "answer": sentence_clean(a_format.format(cause=cause))}
    return result

<div dir="rtl">
    در تابع <code>extract_question</code> ابتدا روی تمامی ساختارهای موجود که امکان طرح سوال از آنها وجود دارد، پیمایشی انجام می‌گیرد. در هر پیمایش، هر ساختار از <code>all_regexes</code> در جمله‌ی ورودی توسط تابع <code>extract_cause_effects</code> بررسی می‌گردد و در صورت داشتن خروجی معتبر، پرسش و پاسخ مربوطه‌ی آن نمایش داده می‌شود.
</div>

In [22]:
def extract_questions(text: str):
    """
    checking all different sentence formats on text. if one is applied ignore others and return
    Parameters
    ----------
    text: str
        input sentence
    
    Returns
    ------
    result: dict
      question and answer sentences.
    """
    for key, value in all_regexes.items():
        result = extract_cause_effects(text, key)
        if result:
            return result
    
sentence = 'رنگین کمونه؛ چون بارون اومده.'
print(extract_questions(sentence_clean(sentence)))

sentence = 'چون بارون اومده، رنگین کمونه.'
print(extract_questions(sentence_clean(sentence)))

{'question': 'چرا رنگین کمونه؟', 'answer': 'زیرا بارون اومده.'}
None
